# 채팅 형식

이 노트북에서는 채팅 형식을 활용하여 특정 작업이나 행동에 맞게 맞춤화되거나 특화된 챗봇과 확장된 대화를 나누는 방법을 살펴봅니다.

## 설정

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [3]:
messages =  [  
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
{'role':'user', 'content':'tell me a joke'},   
{'role':'assistant', 'content':'Why did the chicken cross the road'},   
{'role':'user', 'content':'I don\'t know'}  ]

In [4]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To reach yonder village, good sir.


In [5]:
messages =  [  
# {'role':'system', 'content':'You are friendly chatbot that speaks Korean very fluently. Reply in Korean.'},
{'role':'system', 'content':'You are friendly chatbot that speaks Korean very fluently. '},
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

안녕하세요, 제 이름은 Isa입니다. 반가워요! 무엇이나 도와드릴까요?


In [6]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot that speaks Korean very fluently. Reply in Korean.'},
# {'role':'system', 'content':'You are friendly chatbot that speaks Korean very fluently.'},
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

안녕하세요! 저는 챗봇이기 때문에 당신의 이름을 알지 못합니다. 제가 도와드릴 수 있는 다른 문제가 있다면 언제든지 말씀해주세요!


In [7]:
messages =  [  
# {'role':'system', 'content':'You are friendly chatbot that speaks Korean very fluently. Reply in Korean.'},
# {'role':'system', 'content':'You are friendly chatbot that speaks Korean very fluently.'},
{'role':'system', 'content':'You are friendly Korean chatbot. You should answer in Korean.'},
# {'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

당신의 이름은 이사(Isa)입니다.


# 주문봇
사용자 프롬프트와 어시스턴트 응답 수집을 자동화하여 주문봇을 구축할 수 있습니다. 주문봇은 피자 가게에서 주문을 받습니다. 

In [8]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

In [9]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are Korean OrderBot, an automated service to collect orders for a pizza restaurant. \
You should answer in Korean.\
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, height=300, loading_indicator=True)

In [10]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

Here's a JSON summary of the previous food order:

```
{
  "pizza": {
    "type": "가지 피자",
    "size": "12인치",
    "toppings": [
      "버섯 토핑"
    ],
    "price": 11.95
  },
  "drinks": [
    {
      "type": "생수",
      "size": "중간 크기",
      "price": 5.00
    }
  ],
  "sides": [],
  "total_price": 16.95
}
```

- 피자: 가지 피자, 12인치, 버섯 토핑 추가, 가격 11.95달러
- 음료수: 생수, 중간 크기, 가격 5.00달러
- 사이드: 없음
- 총 가격: 16.95달러


In [11]:
context

[{'role': 'system',
  'content': "\nYou are Korean OrderBot, an automated service to collect orders for a pizza restaurant. You should answer in Korean.You first greet the customer, then collects the order, and then asks if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Finally you collect the payment.Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.You respond in a short, very conversational friendly style. The menu includes pepperoni pizza  12.95, 10.00, 7.00 cheese pizza   10.95, 9.25, 6.50 eggplant pizza   11.95, 9.75, 6.75 fries 4.50, 3.50 greek salad 7.25 Toppings: extra cheese 2.00, mushrooms 1.50 sausage 3.00 canadian bacon 3.50 AI sauce 1.50 peppers 1.00 Drinks: coke 3.00, 2.00, 1.00 sprite 3.00, 2.00, 1.00 bottled water 5.00 "},
 {'role': 'user', 'content': ''},
 {'role': 'assistant

In [12]:
messages

[{'role': 'system',
  'content': "\nYou are Korean OrderBot, an automated service to collect orders for a pizza restaurant. You should answer in Korean.You first greet the customer, then collects the order, and then asks if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Finally you collect the payment.Make sure to clarify all options, extras and sizes to uniquely identify the item from the menu.You respond in a short, very conversational friendly style. The menu includes pepperoni pizza  12.95, 10.00, 7.00 cheese pizza   10.95, 9.25, 6.50 eggplant pizza   11.95, 9.75, 6.75 fries 4.50, 3.50 greek salad 7.25 Toppings: extra cheese 2.00, mushrooms 1.50 sausage 3.00 canadian bacon 3.50 AI sauce 1.50 peppers 1.00 Drinks: coke 3.00, 2.00, 1.00 sprite 3.00, 2.00, 1.00 bottled water 5.00 "},
 {'role': 'user', 'content': ''},
 {'role': 'assistant